# Overall Workflow
## 1. The Dataset
## 2. Preprocessing Before Modeling
## 3. Hyperparameter Tuning - Bayesian Optimization

# Packages

In [1]:
# Data structures
import numpy as np
import pandas as pd

# Data formats
import csv
import json

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sb
plt.style.use("ggplot")

# Statistical testings
from scipy.stats import f
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency

# Data preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine learning algorithms
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import xgboost as xgb 
import lightgbm as lgb

# Hyperparameter tuning
from hyperopt import hp
from hyperopt import Trials
from hyperopt import tpe
from timeit import default_timer as timer
from hyperopt import STATUS_OK
from hyperopt import fmin
from bayes_opt import BayesianOptimization

# Model validation
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

/Users/wuchenhong/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


# 1. The dataset

In [2]:
data = pd.read_csv('data.csv')
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
# After inspecting this column, it's an empty column.
data.drop(columns=['Unnamed: 32','id'], inplace = True)

In [4]:
# 569 data with 31 features.
data.shape

(569, 31)

# 2. Preprocessing Before Modeling
- **2.1 One-hot encoding - response**
- **2.2 Train & test splitting (80%/20%)**
- **2.3 Standardization**
    - Why standardization is needed? -> Regularization purpose.

## 2.1 One-hot encoding - response

In [5]:
data['diagnosis'].head()

0    M
1    M
2    M
3    M
4    M
Name: diagnosis, dtype: object

In [6]:
# One-hot encoding for categorical features containing two levels.
# The response variable, diagnosis, contains two levels, M and B.
le = LabelEncoder()
le.fit(data['diagnosis'])
data['diagnosis'] = le.transform(data['diagnosis'])
data['diagnosis'].head()

0    1
1    1
2    1
3    1
4    1
Name: diagnosis, dtype: int64

In [7]:
# Make sure the transformation is correct!
# 0: B (benign)
# 1: M (malignant)
data['diagnosis'].unique()

array([1, 0])

In [8]:
data['diagnosis'].value_counts() / data.shape[0]

0    0.627417
1    0.372583
Name: diagnosis, dtype: float64

## 2.2 Train & test splitting (80%, 20%)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data[data.columns.difference(['diagnosis'])], 
                                                    data.diagnosis, 
                                                    test_size=0.2, random_state=0) 

In [10]:
# Make sure the splitting is correct!
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(455, 30) (455,)
(114, 30) (114,)


In [11]:
# The label distribution in testing data is also closed to the original distribution.
print(sum(y_test == 0) / len(y_test))
print(sum(y_test == 1) / len(y_test))

0.5877192982456141
0.41228070175438597


## 2.3 Standardization
- Standardization is needed for Regularized Logistic Regression and SGDClassifier.

In [ ]:
# Test data are unseen data! 
# Since my purpose is to build a prediction model, 
# I shouldn't contaminate train and test data together when standardizing them.  
std_scaler = StandardScaler().fit(x_train)

train_standardized_data = std_scaler.transform(x_train)
test_standardized_data = std_scaler.transform(x_test)

In [ ]:
# Make sure the standardization is correct (mean must be 0 and variance must be 1).
pd.DataFrame(train_standardized_data).describe()

# 3. Hyperparameter Tuning - Bayesian Optimization
- **3.1 Logistic Regression**
- **3.2 SGDClassifier**
- **3.3 Decision Tree**
- **3.4 Bagging - Decision Tree**
- **3.5 Random Forest**
- **3.6 AdaBoost**
- **3.7 Gradient Boosting**
- **3.8 XGBoost**
- **3.9 LightGBM**

## 3.1 Logistic Regression
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    - penalty: Used to specify the norm used in the penalization.
    - C: Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
    - fit_intercept: Specifies if a constant (a.k.a. bias or intercept) should be added to the decision function.
    - random_state: The seed of the pseudo random number generator to use when shuffling the data.

In [ ]:
# Define the search space (hyperparameter space).
space = {
    'penalty': hp.choice('penalty', ['l2', 'l1']),
    'C': hp.uniform('C', 0.01, 1000.0),
    'fit_intercept':hp.choice('fit_intercept', [True, False]),
    'random_state':0,
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = LogisticRegression(**hyperparameters)
    cv_results = cross_validate(model, train_standardized_data, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [12]:
def bayesian_optimize(n_eval):
    
    # Record searching results.
    trials = Trials()
    
    # Create a csv file to store results.
    of_connection = open(OUT_FILE, 'w')
    writer = csv.writer(of_connection)

    ITERATION = 0
    
    # Write column names in the file.
    headers = ['loss', 'hyperparameters', 'iteration', 'runtime']
    writer.writerow(headers)
    of_connection.close()
    
    
    # Run x evaluations.
    MAX_EVALS = n_eval

    # Start optimization!
    best = fmin(fn = objective, 
                space = space, 
                algo = tpe.suggest, 
                trials = trials,
                max_evals = MAX_EVALS)

    return best

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Breast_Cancer_LR.csv'

In [ ]:
bayesian_optimize(100)

## 3.2 SGDClassifier
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html
    - loss: 'log' loss gives logistic regression.
    - penalty: Regularization term.
    - alpha: Constant that multiplies the regularization term.
    - l1_ratio: The Elastic Net mixing parameter. l1_ratio=0 corresponds to L2 penalty, l1_ratio=1 to L1.
    - fit_intercept: Whether the intercept should be estimated or not. 
    - random_state: The seed of the pseudo random number generator to use when shuffling the data.

In [ ]:
# Define the search space (hyperparameter space).
space = {
    'loss':'log',
    'penalty': hp.choice('penalty', ['none', 'l2', 'l1', 'elasticnet']),
    'alpha': hp.uniform('alpha', 0.0001, 100.0),
    'l1_ratio': hp.uniform('l1_ratio', 0.0, 1.0),
    'fit_intercept':hp.choice('fit_intercept', [True, False]),
    'random_state':0,
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    """Objective function for multiple linear regression hyperparameter optimization."""
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = SGDClassifier(**hyperparameters)
    cv_results = cross_validate(model, train_standardized_data, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Breast_Cancer_SGD.csv'

In [ ]:
bayesian_optimize(100)

## 3.3 Decision Tree
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
    - criterion: The function to measure the quality of a split.
    - max_depth: The maximum depth of the tree.
    - min_samples_split: The minimum number of samples required to split an internal node.
    - min_samples_leaf: The minimum number of samples required to be at a leaf node. (Will produce a conflict since max_depth determines the number of samples at a leaf node too.)
    - random_state

In [ ]:
# Define the search space (hyperparameter space).
space = {
    'criterion': hp.choice('criterion', ['gini', 'entropy']),
    'max_depth': hp.uniform('max_depth', 1.0, 500.0),
    'min_samples_split': hp.uniform('min_samples_split', 0.0, 1.0),
    'random_state': 0,
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = DecisionTreeClassifier(**hyperparameters)
    cv_results = cross_validate(model, x_train, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Breast_Cancer_DT.csv'

In [ ]:
bayesian_optimize(100)

## 3.4 Bagging - Decision Tree
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html
    - base_estimator: The base estimator to fit on random subsets of the dataset.
    - n_estimators: The number of base estimators in the ensemble.
    - max_samples: The number of samples to draw from X to train each base estimator.
    - bootstrap: Whether samples are drawn with replacement.
    - random_state

In [ ]:
pd.read_csv('Breast_Cancer_DT.csv').sort_values(by='loss').head(2)

In [ ]:
pd.read_csv('Breast_Cancer_DT.csv').sort_values(by='loss').hyperparameters.head(2).values

In [ ]:
# Define the search space (hyperparameter space).

# For bagging, it needs to have a base estimator, thus I chose the top two 
# optimal decision trees and a default decision tree (to perform bagging, 
# it's better to have a 'deeper' tree). 
optimal_dt1 = DecisionTreeClassifier(criterion='entropy',
                                     max_depth=448.2261637557397,
                                     min_samples_split=0.08852350974712808,
                                     random_state=0)

optimal_dt2 = DecisionTreeClassifier(criterion='entropy',
                                     max_depth=448.2261637557397,
                                     min_samples_split=0.08852350974712808,
                                     random_state=0)

default_dt = DecisionTreeClassifier(max_depth=15, random_state=0)

space = {
    'base_estimator': hp.choice('base_estimator', [optimal_dt1, optimal_dt2, default_dt]),
    'n_estimators': hp.randint('n_estimators', 500),
    'max_samples': hp.uniform('max_samples', 0.0, 1.0),
    'bootstrap': hp.choice('bootstrap', [False, True]),
    'random_state': 0,
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = BaggingClassifier(**hyperparameters)
    cv_results = cross_validate(model, x_train, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    #print(hyperparameters)
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Breast_Cancer_Bag_DT.csv'

In [ ]:
bayesian_optimize(100)

## 3.5 Random Forest
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
    - n_estimators: The number of trees in the forest.
    - criterion: The function to measure the quality of a split.
    - max_depth: The maximum depth of the tree.
    - min_samples_split: The minimum number of samples required to split an internal node.
    - min_samples_leaf: The minimum number of samples required to be at a leaf node.
    - max_features: The number of features to consider when looking for the best split.
    - bootstrap: Whether bootstrap samples are used when building trees.

In [ ]:
space = {
    'n_estimators': hp.randint('n_estimators', 500),
    'criterion': hp.choice('criterion', ['gini', 'entropy']),
    'max_depth': hp.randint('max_depth', 500),
    'min_samples_split': hp.uniform('min_samples_split', 0.0, 1.0),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.002, 0.5),
    'max_features': hp.uniform('max_features', 0.0, 1.0),
    'bootstrap': hp.choice('bootstrap', [False, True]),
    'random_state': 0
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = RandomForestClassifier(**hyperparameters)
    cv_results = cross_validate(model, x_train, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    #print(hyperparameters)
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Breast_Cancer_RF.csv'

In [ ]:
bayesian_optimize(100)

## 3.6 AdaBoost
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
    - base_estimator: The base estimator from which the boosted ensemble is built.
    - n_estimators: The maximum number of estimators at which boosting is terminated.
    - learning_rate: Learning rate shrinks the contribution of each classifier by learning_rate. There is a trade-off between learning_rate and n_estimators.
    - random_state

In [ ]:
# Typically boosting requires "stumps" (shallow trees with depth from 4 to 8) to improve
# model performance. 
# For a fair optimization, use the same base learners for following boosting algorithms.
base_1 = DecisionTreeClassifier(max_depth=4,random_state=0)
base_2 = DecisionTreeClassifier(max_depth=5,random_state=0)
base_3 = DecisionTreeClassifier(max_depth=6,random_state=0)
base_4 = DecisionTreeClassifier(max_depth=7,random_state=0)
base_5 = DecisionTreeClassifier(max_depth=8,random_state=0)

In [ ]:
space = {
    'base_estimator': hp.choice('base_estimator', [base_1,base_2,base_3,base_4,base_5]),
    'n_estimators': hp.randint('n_estimators', 500),
    'learning_rate': hp.uniform('learning_rate', 0.0, 1.0),
    'random_state': 0
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = AdaBoostClassifier(**hyperparameters)
    cv_results = cross_validate(model, x_train, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    #print(hyperparameters)
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Breast_Cancer_AdaBoost.csv'

In [ ]:
bayesian_optimize(100)

## 3.7 Gradient Boosting
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
    - n_estimators: The number of boosting stages to perform.
    - max_depth: maximum depth of the individual regression estimators.
    - subsample: The fraction of samples to be used for fitting the individual base learners.
    - max_features: The number of features to consider when looking for the best split.
    - learning_rate: learning rate shrinks the contribution of each tree by learning_rate. There is a trade-off between learning_rate and n_estimators.
    - random_state

In [ ]:
space = {
    'n_estimators': hp.randint('n_estimators', 500),
    'max_depth': hp.uniform('max_depth', 0, 8),
    'subsample': hp.uniform('subsample', 0.0, 1.0),
    'max_features': hp.uniform('max_features', 0.0, 1.0),
    'learning_rate': hp.uniform('learning_rate', 0.0, 1.0),
    'random_state': 0
}

In [ ]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    model = GradientBoostingClassifier(**hyperparameters)
    cv_results = cross_validate(model, x_train, y_train, 
                                scoring='roc_auc', cv=5, 
                                return_train_score=False)
    run_time = timer() - start
    #print(hyperparameters)
    
    # Loss must be minimized (put a negative sign)
    loss = -(np.mean(cv_results['test_score']))

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Breast_Cancer_GradientBoost.csv'

In [ ]:
bayesian_optimize(100)

## 3.8 XGBoost
- Reference: https://xgboost.readthedocs.io/en/latest/parameter.html
    - Parameters for Tree Booster
        - eta: Step size shrinkage used in update to prevents overfitting.
        - max_depth: Maximum depth of a tree.
        - subsample: Subsample ratio of the training instances.
        - colsample_bytree: Subsample ratio of columns when constructing each tree.
        - colsample_bylevel: Subsample ratio of columns for each level.
        - colsample_bynode: Subsample ratio of columns for each node (split).
        - lambda: L2 regularization term on weights.
        - alpha: L1 regularization term on weights.
    - Learning Task Parameters
        - objective: binary:logistic (logistic regression for binary classification, output probability).
        - eval_metric: auc (Area under the curve).
    - Command Line Parameters
        - num_round: The number of rounds for boosting.
        - task: train (training using data).

In [13]:
space = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'eta': hp.uniform('eta', 0.0, 1.0),
    'max_depth': hp.randint('max_depth', 8),
    'subsample': hp.uniform('subsample', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.0, 1.0),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.0, 1.0),
    'colsample_bynode': hp.uniform('colsample_bynode', 0.0, 1.0),
    'lambda': hp.uniform('lambda', 0.0, 1.0),
    'alpha': hp.uniform('alpha', 0.0, 1.0),
    'num_round': hp.randint('num_round', 500),
    'task': 'train'
}

In [14]:
# Define the objective for the optimization.
# Here is to minimize the negative roc_auc (area under the curve) score since roc_auc 
# is a metric that gives a selection between precision and recall.
def objective(hyperparameters):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    start = timer()
    
    # Perform 5-fold cross validation
    cv = xgb.cv(hyperparameters, xgb_train, num_boost_round=hyperparameters['num_round'], 
                nfold=5, early_stopping_rounds=10, verbose_eval=0)
    #print(cv)
    
    run_time = timer() - start
    
    # Loss must be minimized (put a negative sign)
    loss = -(cv['test-auc-mean'][cv.shape[0]-1])
    
    # Update the number of boosting rounds once the early stopping is finished.
    hyperparameters['num_round'] = cv.shape[0]

    # Write searching results to a csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time])
    of_connection.close()

    # Dictionary with information for later evaluations
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [15]:
xgb_train = xgb.DMatrix(x_train, label=y_train)

In [16]:
# Global variable
global ITERATION

ITERATION = 0

OUT_FILE = 'Breast_Cancer_XGB.csv'

In [17]:
bayesian_optimize(100)

[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=5
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=5
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=5
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=5
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=4
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=4
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_d

[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=4
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[

[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[16:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_d

[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[

[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[

...

[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=3
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=4
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=4
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=3

[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=6
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=5
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=5
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=5
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=5
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=4
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=4
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_

[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[16:54:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=5
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=4
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=

[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[16:54:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[

{'alpha': 0.4369943595381558,
 'colsample_bylevel': 0.2905096078115737,
 'colsample_bynode': 0.6300404451387125,
 'colsample_bytree': 0.5550946683724962,
 'eta': 0.3392632704567006,
 'lambda': 0.41081849491034733,
 'max_depth': 5,
 'num_round': 221,
 'subsample': 0.5315661431503793}

## 3.9 LightGBM
- Reference:
    - Core parameters
        - task: default = train.
        - objective: binary, binary log loss classification (or logistic regression).
        - boosting: default = gbdt (traditional Gradient Boosting Decision Tree).
        - data: path of training data, LightGBM will train from this data.
        - num_iterations: number of boosting iterations.
        - learning_rate: shrinkage rate.
        - num_leaves: max number of leaves in one tree
        - seed: this seed is used to generate other seeds.
    - Learning Control Parameters
        - max_depth: limit the max depth for tree model. 
        - bagging_fraction: randomly select part of data without resampling.
        - bagging_freq: 0 means disable bagging; k means perform bagging at every k iteration.
        - feature_fraction: randomly select part of features on each iteration.
        - early_stopping_round: will stop training if one metric of one validation data doesn't improve in last early_stopping_round rounds.
        - lambda_l1: L1 regularization.
        - lambda_l2: L2 regularization.
    - Metric Parameters
        - metric: metric(s) to be evaluated on the evaluation set(s).

In [18]:
def bayes_parameter_opt_lgb(X, y, file_path, init_round=15, opt_round=25, n_folds=5, 
                            random_seed=0, output_process=False):
    # Prepare data
    train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    
    # Hyperparameters
    def lgb_eval(num_iterations, learning_rate, max_depth,
                 bagging_fraction, feature_fraction, 
                 lambda_l1, lambda_l2):
        # Predefined parameters
        params = {
            "objective":"binary", "seed":0, "metric":"auc", "bagging_freq":1
        }
        # Hyperparameters
        params['num_iterations'] = int(round(num_iterations))
        params['learning_rate'] = learning_rate
        params['max_depth'] = int(round(max_depth))
        params['bagging_fraction'] = bagging_fraction
        params['feature_fraction'] = feature_fraction
        params['lambda_l1'] = lambda_l1
        params['lambda_l2'] = lambda_l2
        
        # 5-fold cross validations
        cv_result = lgb.cv(params, train_set=train_data, 
                           num_boost_round=num_iterations,
                           nfold=n_folds, 
                           early_stopping_rounds=10,
                           seed=random_seed, 
                           verbose_eval=200, 
                           stratified=False)
        
        # Update the number of boosting rounds once the early stopping is finished.
        params['num_iterations'] = len(cv_result['auc-mean'])
        
        return cv_result['auc-mean'][-1]
    
    
    # Hyperparameter range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_iterations': (100, 500),
                                            'learning_rate': (0.1, 1.0),
                                            'max_depth': (1, 8),
                                            'bagging_fraction': (0.1, 1.0),
                                            'feature_fraction': (0.1, 1.0),
                                            'lambda_l1': (0.0, 1.0),
                                            'lambda_l2': (0.0, 1.0)}, 
                                 random_state=0)
    
    # Start optimizing (maximize the auc score)
    lgbBO.maximize(init_points=init_round, n_iter=opt_round,acq='ei')

    # Output optimization process
    if output_process==True: 
        # Write searching results to a csv file ('a' means append)
        of_connection = open(file_path, 'a')
        writer = csv.writer(of_connection)
        headers = ['loss', 'hyperparameters', 'iteration']
        writer.writerow(headers)
        for i in range(len(lgbBO.res)):
            writer.writerow([
                lgbBO.res[i]['target'], lgbBO.res[i]['params'], i])
        of_connection.close()
        
    # Return the searching result
    return lgbBO.res

In [19]:
# In total, performed 100 times of optimization.
opt_params = bayes_parameter_opt_lgb(X=x_train, y=y_train, 
                                     file_path='Breast_Cancer_LightGBM.csv',
                                     init_round=10, opt_round=90, n_folds=5, 
                                     random_seed=0, output_process=True)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | num_it... |
-------------------------------------------------------------------------------------------------------------


/Users/wuchenhong/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:426: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  1        |  0.9897   |  0.5939   |  0.7437   |  0.6028   |  0.5449   |  0.4813   |  5.521    |  275.0    |
|  2        |  0.9895   |  0.9026   |  0.9673   |  0.3834   |  0.7917   |  0.576    |  4.976    |  470.2    |


/Users/wuchenhong/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:426: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/wuchenhong/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:426: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  3        |  0.9865   |  0.1639   |  0.1784   |  0.02022  |  0.8326   |  0.8003   |  7.09     |  491.4    |
|  4        |  0.9901   |  0.8192   |  0.5153   |  0.7805   |  0.1183   |  0.6759   |  2.003    |  477.9    |
|  5        |  0.9862   |  0.5697   |  0.4732   |  0.2646   |  0.7742   |  0.5105   |  4.979    |  107.5    |


...

|  98       |  0.9888   |  1.0      |  0.1      |  0.0      |  0.0      |  0.1      |  8.0      |  120.2    |


/Users/wuchenhong/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:426: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  99       |  0.9875   |  1.0      |  0.1      |  0.0      |  0.0      |  0.1      |  3.109    |  104.9    |


/Users/wuchenhong/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:426: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  100      |  0.99     |  1.0      |  0.1      |  0.0      |  0.0      |  1.0      |  4.456    |  114.8    |
